In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

C:\Users\Richie Rozario\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

In [3]:
import pandas as pd
tweet = pd.read_csv("h.csv", encoding="ISO-8859-1")

In [4]:
tweet.shape

(19001, 4)

In [5]:
tweet.head(10)

,content,annotation__notes,annotation__label__-,extras
0,I hate .... you .... zzzzzzzz,NaN,1,NaN
1,Hey... Stop twitting and post a damn blog!,NaN,1,NaN
2,http://twitpic.com/qtt0 - O.O bingo is waaaaay...,NaN,1,NaN
3,xD! i know. hmm ....i wonder how it feels like...,NaN,1,NaN
4,oh youuuu little bitch. Just keep your winter ...,NaN,1,NaN
5,not to sound amazingly astoundingly gay (not ...,NaN,1,NaN
6,as much as i hate auto-DM's i also check out ...,NaN,1,NaN
7,I'm blocking you because you have no value. Go...,NaN,1,NaN
8,If only Edward and Bella were slightly less em...,NaN,1,NaN
9,@scanman and @asthepumpturns. Yeah I am a mea...,NaN,1,NaN


In [6]:
tweet.rename(columns = {"annotation__label__-": "label", "content": "tweet"}, inplace = True)
tweet.drop(['annotation__notes'], axis = 1, inplace = True)
tweet.drop(['extras'], axis = 1, inplace = True)
tweet.head(10)

,tweet,label
0,I hate .... you .... zzzzzzzz,1
1,Hey... Stop twitting and post a damn blog!,1
2,http://twitpic.com/qtt0 - O.O bingo is waaaaay...,1
3,xD! i know. hmm ....i wonder how it feels like...,1
4,oh youuuu little bitch. Just keep your winter ...,1
5,not to sound amazingly astoundingly gay (not ...,1
6,as much as i hate auto-DM's i also check out ...,1
7,I'm blocking you because you have no value. Go...,1
8,If only Edward and Bella were slightly less em...,1
9,@scanman and @asthepumpturns. Yeah I am a mea...,1


In [7]:
tweet.label.value_counts()

0    11162
1     7839
Name: label, dtype: int64

In [8]:
tweet['tweet'] = tweet['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
tweet['tweet'].head()

0                        i hate .... you .... zzzzzzzz
1           hey... stop twitting and post a damn blog!
2    http://twitpic.com/qtt0 - o.o bingo is waaaaay...
3    xd! i know. hmm ....i wonder how it feels like...
4    oh youuuu little bitch. just keep your winter ...
Name: tweet, dtype: object

In [9]:
tweet['tweet'] = tweet['tweet'].str.replace('[^\w\s]','')
tweet['tweet'] = tweet['tweet'].str.replace(r'_','')
tweet['tweet'].head()

0                                i hate  you  zzzzzzzz
1               hey stop twitting and post a damn blog
2    httptwitpiccomqtt0  oo bingo is waaaaay to har...
3    xd i know hmm i wonder how it feels like to fu...
4    oh youuuu little bitch just keep your winter g...
Name: tweet, dtype: object

In [10]:
tweet['numerics'] = tweet['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
tweet[['tweet','numerics']].head(10)

,tweet,numerics
0,i hate you zzzzzzzz,0
1,hey stop twitting and post a damn blog,0
2,httptwitpiccomqtt0 oo bingo is waaaaay to har...,0
3,xd i know hmm i wonder how it feels like to fu...,0
4,oh youuuu little bitch just keep your winter g...,0
5,not to sound amazingly astoundingly gay not th...,0
6,as much as i hate autodms i also check out lin...,0
7,im blocking you because you have no value go b...,0
8,if only edward and bella were slightly less emo,0
9,scanman and asthepumpturns yeah i am a mean me...,0


In [11]:
tweet['tweet'] = tweet['tweet'].str.replace('[\d+]','')
tweet['tweet'].head()

0                                i hate  you  zzzzzzzz
1               hey stop twitting and post a damn blog
2    httptwitpiccomqtt  oo bingo is waaaaay to hard...
3    xd i know hmm i wonder how it feels like to fu...
4    oh youuuu little bitch just keep your winter g...
Name: tweet, dtype: object

In [12]:
stop = stopwords.words('english')
tweet['tweet'] = tweet['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
tweet['tweet'].head()

0                                        hate zzzzzzzz
1                     hey stop twitting post damn blog
2    httptwitpiccomqtt oo bingo waaaaay hard hate w...
3    xd know hmm wonder feels like fuck bingo hard ...
4    oh youuuu little bitch keep winter gloves away...
Name: tweet, dtype: object

In [13]:
tweet['tweet'] = tweet['tweet'].apply(lambda x: " ".join(x for x in x.split() if len(x) > 2))
tweet['tweet'].head()

0                                        hate zzzzzzzz
1                     hey stop twitting post damn blog
2    httptwitpiccomqtt bingo waaaaay hard hate wann...
3    know hmm wonder feels like fuck bingo hard hmm...
4    youuuu little bitch keep winter gloves away zi...
Name: tweet, dtype: object

In [14]:
freq = pd.Series(' '.join(tweet['tweet']).split()).value_counts()[:10]
freq = list(freq.index)
tweet['tweet'] = tweet['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
tweet.head(10)

,tweet,label,numerics
0,zzzzzzzz,1,0
1,hey stop twitting post blog,1,0
2,httptwitpiccomqtt bingo waaaaay hard wanna itw...,1,0
3,know hmm wonder feels bingo hard hmmmm guss il...,1,0
4,youuuu little bitch keep winter gloves away zi...,1,0
5,sound amazingly astoundingly gay thats bad sou...,1,0
6,much autodms also check links theyre music rel...,1,0
7,blocking value back telemarketing loser market...,1,0
8,edward bella slightly less emo,1,0
9,scanman asthepumpturns yeah mean mean person h...,1,0


In [15]:
from sklearn.cross_validation import train_test_split

X = tweet.tweet
y = tweet.label

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(13300,)
(5701,)
(13300,)
(5701,)


C:\Users\Richie Rozario\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
def tokenize(tweet):
    words = word_tokenize(tweet)
    
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    lemma = WordNetLemmatizer()
    words = [lemma.lemmatize(word) for word in words]
    
    return words

In [17]:
vect = CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        encoding='ISO-8859-1', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=tokenize, vocabulary=None)
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='ISO-8859-1',
        input='content', lowercase=True, max_df=1.0, max_features=None,
        min_df=1, ngram_range=(1, 2), preprocessor=None,
        stop_words='english', strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x000001CB60C8EA60>,
        vocabulary=None)

In [18]:
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<13300x51746 sparse matrix of type '<class 'numpy.int64'>'
	with 126916 stored elements in Compressed Sparse Row format>

In [19]:
X_test_dtm = vect.transform(X_test)
print(X_test_dtm)

  (0, 3320)	1
  (0, 9959)	1
  (0, 10075)	1
  (0, 13625)	1
  (0, 24956)	1
  (0, 29065)	1
  (0, 32303)	1
  (0, 50504)	2
  (1, 5741)	1
  (1, 12080)	1
  (1, 14009)	1
  (1, 14056)	1
  (1, 32515)	1
  (1, 32516)	1
  (1, 44047)	1
  (1, 44088)	1
  (2, 20587)	1
  (2, 27537)	1
  (3, 11672)	1
  (3, 11673)	1
  (3, 12524)	1
  (3, 12526)	1
  (3, 15682)	1
  (3, 15699)	1
  (3, 17667)	1
  :	:
  (5694, 21328)	1
  (5694, 42342)	1
  (5694, 44879)	1
  (5694, 45901)	1
  (5695, 853)	1
  (5695, 11478)	1
  (5695, 14009)	1
  (5697, 30396)	1
  (5697, 37660)	1
  (5697, 37697)	1
  (5697, 37965)	1
  (5697, 41929)	1
  (5697, 47048)	1
  (5698, 132)	1
  (5698, 3094)	1
  (5698, 9994)	1
  (5698, 20963)	1
  (5698, 30574)	1
  (5698, 35470)	1
  (5698, 42561)	1
  (5698, 43358)	1
  (5698, 46384)	1
  (5699, 3647)	2
  (5699, 14831)	1
  (5699, 41429)	1


In [20]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [21]:
%time nb.fit(X_train_dtm, y_train)

Wall time: 225 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_pred_class = nb.predict(X_test_dtm)

In [23]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.8433608138922996

In [28]:
from keras.models import Sequential
from keras import layers

input_dim = X_train_dtm.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [30]:
x_val = X_train_dtm[:4300]
partial_x_train = X_train_dtm[4300:]

y_val = y_train[:4300]
partial_y_train = y_train[4300:]

In [31]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 9000 samples, validate on 4300 samples
Epoch 1/40
9000/9000 [==============================] - 11s 1ms/step - loss: 0.6838 - acc: 0.6207 - val_loss: 0.6704 - val_acc: 0.6440
Epoch 2/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.6358 - acc: 0.7098 - val_loss: 0.6422 - val_acc: 0.7012
Epoch 3/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.5803 - acc: 0.8000 - val_loss: 0.6140 - val_acc: 0.7379
Epoch 4/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.5239 - acc: 0.8537 - val_loss: 0.5871 - val_acc: 0.7735
Epoch 5/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.4707 - acc: 0.8903 - val_loss: 0.5620 - val_acc: 0.7891
Epoch 6/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.4224 - acc: 0.9106 - val_loss: 0.5397 - val_acc: 0.7986
Epoch 7/40
9000/9000 [==============================] - 10s 1ms/step - loss: 0.3796 - acc: 0.9232 - val_loss: 0.5198 - val_acc: 0.8081
Epoch 8

In [34]:
results = model.evaluate(X_test_dtm, y_test)

print(results)

5701/5701 [==============================] - 4s 760us/step
[0.427194032098636, 0.8168742325907735]
